In [1]:
import os
os.chdir('../scripts')

from scrape_domain import *
from constants import *
from utils import *

In [2]:
#find pattern for week/annual rent
year_pattern = r"p.a.|annually|annual"
week_pattern = r"pw|week"

regex_week_pattern = re.compile(week_pattern)
regex_year_pattern = re.compile(year_pattern)


def remove_chars(s):
    return re.sub('[^0-9.]+', '', s) 

#flag type of rent with 1(weekly), 0(annul), None(unknown)
def type_flag(s):
    # week rent
    if (len(regex_week_pattern.findall(s.lower())) != 0 )& (len(regex_year_pattern.findall(s.lower())) == 0) :
        return 1
    # year rent
    elif (len(regex_week_pattern.findall(s.lower())) == 0) & (len(regex_year_pattern.findall(s.lower())) != 0) :
        return 0
    # not speficied
    else:
        return None

#convert rent to weekly rent
#if not sure rent type, it will be considered as weekly rent
def convert_rent(s):
    # week rent
    if (len(regex_week_pattern.findall(s.lower())) != 0 )& (len(regex_year_pattern.findall(s.lower())) == 0) :
        return remove_chars(s)
    # year rent
    elif (len(regex_week_pattern.findall(s.lower())) == 0) & (len(regex_year_pattern.findall(s.lower())) != 0) :
        return float(remove_chars(s))//12
    # not speficied
    else:
        return remove_chars(s)

In [3]:

#input url return dict
def input_url_output_df(url_list):
    basic_feature_list = []
    for url in url_list  :
        
        link = url
        
        # opening urls
        bs_object = BeautifulSoup(requests.get(link, headers=headers).text, "html.parser")

        #recent rented
        recent_rented_number = bs_object.find("div", {"class": "css-12uo2x5"})

        #neighbourhood family vs single ratio
        neighb_family = bs_object.find("span", {"class": "css-1ruej2h"})
        neighb_single = bs_object.find("span", {"class": "css-1ymya4c"})


        #street owner vs renter
        street_renter = bs_object.find("span", {"class": "css-1ejmyle"})
        street_owner = bs_object.find("span", {"class": "css-1jhlvvv"})


        #long term resident percentage
        long_term_resi= bs_object.find("div", {"class": "css-ibsnk8"})

        # extracting property price
        property_price = bs_object.find("div", {"data-testid": "listing-details__summary-title"})



        # dictionary to store temporary data
        basic_feature_dict = {}
        pattern1 = re.compile(r'>(.+)<.')

        # putting 'none' if recent_rented number is missing   
        if recent_rented_number is None:
            basic_feature_dict['recent_rented_number'] = None

        else:
            basic_feature_dict['recent_rented_number'] = pattern1.findall(str(recent_rented_number))[0]



        # putting 'none' if longterm resident number is missing   
        if long_term_resi is None:
            basic_feature_dict['long_term_resi_percentage'] = None

        else:
            basic_feature_dict['long_term_resi_percentage']  = float(str(long_term_resi.text[:-1]))/100



        # putting 'none' if neighbourhood family/single number is missing   
        if (neighb_family is None) |(neighb_single is None) :
            basic_feature_dict['neighb_family_single_ratio'] = None

        else:
            #to avoid zero division
            add_on = 0.000001
            basic_feature_dict['neighb_family_single_ratio'] = float(str(neighb_family.text[:-1]))/(float(str(neighb_single.text[:-1]))+add_on)




        # putting 'none' if street owener/renter number is missing   
        if (street_owner is None) | (street_renter is None):
            basic_feature_dict['street_renter_owner_ratio'] = None

        else:
            #to avoid zero division
            add_on = 0.000001
            basic_feature_dict['street_renter_owner_ratio'] = float(str(street_renter.text[:-1]))/(float(str(street_owner.text[:-1]))+add_on)



        # putting 'none' if price is missing    
        if property_price is None:
            basic_feature_dict['price'] = None

        else:
            basic_feature_dict['price'] = pattern1.findall(str(property_price))[0]
            
         # appending all the data into a list
        basic_feature_list.append(basic_feature_dict)
        
    return basic_feature_list
        
        

In [4]:
def list_to_df(basic_feature_list):
    
    recent_rented_number_list = []
    street_renter_owner_ratio_list = []
    neighb_family_single_ratio_list = []
    long_term_resi_percentage_list =[]
    price_list = []

    for row in basic_feature_list:
        # checking if the row cointains 'price'         
        if 'price' in row:
            price_list.append(row['price'])
        else:
            price_list.append(None)        

        # checking if the row cointains 'recent_rented_number'            
        if 'recent_rented_number' in row:
            recent_rented_number_list.append(row['recent_rented_number'])
        else:
            recent_rented_number_list.append(None)


        # checking if the row cointains 'long_term_resi_percentage'            
        if 'long_term_resi_percentage' in row:
            long_term_resi_percentage_list.append(row['long_term_resi_percentage'])
        else:
            long_term_resi_percentage_list.append(None)

        # checking if the row cointains 'street_renter_owner_ratio'            
        if 'street_renter_owner_ratio' in row:
            street_renter_owner_ratio_list.append(row['street_renter_owner_ratio'])
        else:
            street_renter_owner_ratio_list.append(None)

        # checking if the row cointains 'neighb_family_single_ratio'            
        if 'neighb_family_single_ratio' in row:
            neighb_family_single_ratio_list.append(row['neighb_family_single_ratio'])
        else:
            neighb_family_single_ratio.append(None)
        
        
    house_dict = {}    
    house_dict['Rent'] = price_list
    house_dict['street_renter_owner_ratio'] = street_renter_owner_ratio_list 
    house_dict['neighb_family_single_ratio'] = neighb_family_single_ratio_list
    house_dict['long_term_resi_percentage'] = long_term_resi_percentage_list
    house_df = pd.DataFrame(house_dict)
    
    house_df['Converted_Rent'] = [convert_rent(i) for i in price_list]
    house_df['Type'] = [type_flag(i) for i in price_list]
    return house_df



In [5]:
vic_postcodes = constants.postcodes['VIC_sample']

domain_links = set()

for i in tqdm(range(len(vic_postcodes))):
    current_links = domain_property_links(vic_postcodes[i])
    domain_links = domain_links | current_links
    
    print(len(current_links))

0it [00:00, ?it/s]/100 [00:00<?, ?it/s]
  1%|          | 1/100 [00:00<00:27,  3.57it/s]

0


0it [00:00, ?it/s]
  2%|▏         | 2/100 [00:00<00:24,  3.93it/s]

0


0it [00:00, ?it/s]
  3%|▎         | 3/100 [00:00<00:32,  2.96it/s]

...

0


0it [00:00, ?it/s]
 99%|█████████▉| 99/100 [01:12<00:00,  1.80it/s]

0


0it [00:00, ?it/s]
100%|██████████| 100/100 [01:12<00:00,  1.37it/s]

0


In [6]:
with open('2022-09-05.VIC.domain.pickle', 'wb') as file:
    pickle.dump(domain_links, file , protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
with open('2022-09-05.VIC.domain.pickle', 'rb') as file:
    links = pickle.load(file)

len(links)

629

In [8]:
links

{'https://www.domain.com.au/0-cnr-watson-rd-mcpherson-rd-leongatha-vic-3953-15592673',
 'https://www.domain.com.au/1-1-haven-court-cranbourne-vic-3977-16066375',
 'https://www.domain.com.au/1-10-bay-street-port-albert-vic-3971-9745101',
 'https://www.domain.com.au/1-12-william-street-cranbourne-vic-3977-16063928',
 'https://www.domain.com.au/1-14-meikle-street-meeniyan-vic-3956-16004995',
 'https://www.domain.com.au/1-15-roadknight-street-lakes-entrance-vic-3909-16061234',
 'https://www.domain.com.au/1-16-kent-street-mornington-vic-3931-16061709',
 'https://www.domain.com.au/1-16-mckitterick-street-meeniyan-vic-3956-15920145',
 'https://www.domain.com.au/1-171-comers-road-kalimna-west-vic-3909-16072320',
 'https://www.domain.com.au/1-18-bayport-drive-langwarrin-vic-3910-16072133',
 'https://www.domain.com.au/1-18-johnson-street-capel-sound-vic-3940-16039887',
 'https://www.domain.com.au/1-18-stawell-street-cranbourne-vic-3977-16032807',
 'https://www.domain.com.au/1-181-dromana-parade-

In [9]:
test_links = links

In [10]:
output_list = input_url_output_df(test_links)

In [11]:
df = list_to_df(output_list)

In [12]:
df

,Rent,street_renter_owner_ratio,neighb_family_single_ratio,long_term_resi_percentage,Converted_Rent,Type
0,$850 per week,0.052632,6.142857,0.65,850,1.0
1,$450,0.250000,1.439024,0.60,450,NaN
2,$360.00 pw,0.250000,2.571428,0.70,360.00,1.0
3,$600 per week,0.052632,7.333333,0.73,600,1.0
4,UNDER APPLICATION Pending Commencement Date.,0.176471,2.030303,0.67,.,NaN
...,...,...,...,...,...,...
624,Under Application,0.250000,2.846154,0.57,,NaN
625,$580 wk,NaN,5.250000,0.77,580,NaN
626,$575.00,NaN,11.499999,0.68,575.00,NaN
627,$420 per week,0.428571,4.555555,NaN,420,1.0
